# **Quantum Leap**

*Solved for 463 points*

### Prompt:
    
    > My friend took the quantum leap and purchased a quantum computer with two qubits. They mentioned using a quantum logic gate to input the flag and they gave me the computers output. I have been stuck and Can NOT figure out the flag.

### Additional Info Provided:

* **1 Text file**, `output`, containing only the following string:

    > wxqvn$Zae${deyZv$d"i

### Initial Thoughts:

I'm guessing the Quantum Computing course from last semester might come in handy here! The main keywords I'm initially seeing in the prompt are `two qubits`, and that the the input flag went through a `quantum logic gate`. 


> In a terribly explained nutshell, qubits are just bits that occasionally have [special properties](https://www.geeksforgeeks.org/differnce-between-bits-and-quantum-bits/) like being able to represent a superposition of 0 and 1 at the same time. Similar to regular circuits of logic gates that can used to represet and manipulate digital bits (AND, OR, NOT, XOR, etc.), there are a number of single and multi qubit quantum logic gates that take advantage of unique properties of qubits (X, Y, Z, Hadamard, SWAP, etc.). That said, understanding exactly how qubits work within a quantum computer is definitely out of the scope of the problem--this seems like some sort of message decoding problem. 


So, after attempting to find my past class notes (and failing), I looked up `two qubit quantum gates` on Google to see how someone might be able to encode a secret message using two qubits:


![](https://i.imgur.com/KKOG7SR.png)


I get this Wikipedia article, which has a brief blurb about how **quantum logic gates are reversible** as well as a diagram of a few qubit gates and their operations. So however this text was encoded, the quantum logic gate was probably applied to the binary and you can apply the same one again to reverse the process. 

### How I solved it:

I ran `xxd -b output` to convert the ASCII text file into binary and grabbed the first byte or so (`01110111`) from the output to try a handful of the two qubit gates informally. It was about at this point that I decided to reread the prompt and finally noticed the other hint: "I have been stuck and **Can NOT** figure out the flag."

On first glance, I initially noticed NOT was capitalized, maybe hinting at a classical NOT gate? 

I didn't see for quiiiiite a while that the C was capital as well. 

Can NOT = CNOT = [a simple 2 qubit quantum gate](https://en.wikipedia.org/wiki/Controlled_NOT_gate)

![](https://i.imgur.com/GaeFg3B.png)

After manually going through the first byte or two of the string by flipping every second bit if the preceding even bit is a 1, I saw that the ASCII `w`, or `01110111`, turned into an ASCII `f`, or `01100110`. 

**Success!**

Now to build a Python script so that I don't have to do the rest manually


### Let's first convert all of the ASCII to a bit string to make it easier to work with!

In [6]:
# File contents of `output`: wxqvn$Zae${deyZv$d"i
 
# Initializing the ASCII string
Text = 'wxqvn$Zae${deyZv$d"i'

# Now let's throw some functions from online in here that convert between ASCII and binary
# Source: https://stackoverflow.com/a/7397689
import binascii

def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int(binascii.hexlify(text.encode(encoding, errors)), 16))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return int2bytes(n).decode(encoding, errors)

def int2bytes(i):
    hex_string = '%x' % i
    n = len(hex_string)
    return binascii.unhexlify(hex_string.zfill(n + (n & 1)))
 
# Getting the Binary value
binary = text_to_bits(Text)
print(binary)

0111011101111000011100010111011001101110001001000101101001100001011001010010010001111011011001000110010101111001010110100111011000100100011001000010001001101001


### Now we have the binary, so let's apply the CNOT gate to it to get back the original input

In [7]:
i = 0
cnot_binary = ''

# Applies 2 bit CNOT operation on a string of bits (flips every second bit if the first bit is a 1)
while i < len(binary):

    # Add the even bits since they're never flipped
    cnot_binary += binary[i]

    # Then flip next bit if the first one is a 1
    if binary[i] == '1':
        if binary[i+1] == '0':
                cnot_binary += '1'
        else:
            cnot_binary += '0'
    else:
        cnot_binary += binary[i+1]
    
    # Checking every other bit
    i+=2

print(cnot_binary)

# Sanity check that the string lengths are the same
print(str(len(binary)) + " bits, " + str(len(cnot_binary)) + " after cnot")

0110011001101100011000010110011101111011001101000101111101110001011101010011010001101110011101000111010101101101010111110110011100110100011101000011001101111101
160 bits, 160 after cnot


### And finally, let's convert back from bits to ASCII using the text_from_bits function we found above!

In [8]:
print(text_from_bits(cnot_binary))

flag{4_qu4ntum_g4t3}
